In [57]:
import pandas as pd
from fastprogress.fastprogress import master_bar, progress_bar

In [3]:
df = pd.read_csv('data/small_corpus.csv')

In [56]:
!pip install fastprogress -q

You should consider upgrading via the '/opt/conda/bin/python -m pip install --upgrade pip' command.


In [4]:
df.head()

,Unnamed: 0,ratings,reviews
0,122468,1,Recently UBISOFT had to settle a huge class-ac...
1,495370,1,"code didn't work, got me a refund."
2,214717,1,"these do not work at all, all i get is static ..."
3,231734,1,well let me start by saying that when i first ...
4,154709,1,"Dont waste your money, you will just end up us..."


In [6]:
def score_to_Target(value):
    if value >= 5:
        return 2
    if value <= 4 and value >= 2:
        return 1
    else:
        return 0
df['labels'] = df['ratings'].apply(lambda x:score_to_Target(x))

In [11]:
df.dropna(subset = ["reviews"], inplace=True)

In [12]:
sentences = df.reviews.values
labels = df.labels.values

In [13]:
from transformers import BertTokenizer

# Load the BERT tokenizer.
print('Loading BERT tokenizer...')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

Loading BERT tokenizer...


In [14]:
max_len = 0

# For every sentence...
for sent in sentences:

    # Tokenize the text and add `[CLS]` and `[SEP]` tokens.
    input_ids = tokenizer.encode(sent, add_special_tokens=True)

    # Update the maximum sentence length.
    max_len = max(max_len, len(input_ids))

print('Max sentence length: ', max_len)

Token indices sequence length is longer than the specified maximum sequence length for this model (667 > 512). Running this sequence through the model will result in indexing errors


Max sentence length:  6767


In [16]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [18]:
input_ids = []
attention_mask = []
for sent in sentences:
    token_dict = tokenizer.encode_plus(
        sent,
        add_special_tokens=True,
        max_length=128,
        padding = 'max_length',
        return_attention_mask=True,
        return_tensors='pt',
        truncation=True
    )
    input_ids.append(token_dict['input_ids'])
    attention_mask.append(token_dict['attention_mask'])

input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_mask, dim=0)
labels = torch.tensor(labels)

print('Original: ', sentences[1])
print('Token IDs:', input_ids[1])

Original:  code didn't work, got me a refund.
Token IDs: tensor([  101,  3642,  2134,  1005,  1056,  2147,  1010,  2288,  2033,  1037,
        25416,  8630,  1012,   102,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
       

<ipython-input-18-694ca8ba0388>:18: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(labels)


In [20]:
from torch.utils.data import TensorDataset,random_split
dataset = TensorDataset(input_ids,attention_masks,labels)

In [21]:
train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size

In [22]:
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

In [24]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

# The DataLoader needs to know our batch size for training, so we specify it 
# here. For fine-tuning BERT on a specific task, the authors recommend a batch 
# size of 16 or 32.
batch_size = 32

# Create the DataLoaders for our training and validation sets.
# We'll take training samples in random order. 
train_dataloader = DataLoader(
            train_dataset,  # The training samples.
            sampler = RandomSampler(train_dataset), # Select batches randomly
            batch_size = batch_size # Trains with this batch size.
        )

# For validation the order doesn't matter, so we'll just read them sequentially.
validation_dataloader = DataLoader(
            val_dataset, # The validation samples.
            sampler = SequentialSampler(val_dataset), # Pull out batches sequentially.
            batch_size = batch_size # Evaluate with this batch size.
        )

In [80]:
from transformers import BertForSequenceClassification, AdamW, BertConfig

# Load BertForSequenceClassification, the pretrained BERT model with a single 
# linear classification layer on top. 
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased", # Use the 12-layer BERT model, with an uncased vocab.
    num_labels = 3, # The number of output labels--2 for binary classification.
                    # You can increase this for multi-class tasks.   
    output_attentions = False, # Whether the model returns attentions weights.
    output_hidden_states = False, # Whether the model returns all hidden-states.
)

# Tell pytorch to run this model on the GPU.
model.cuda()
optimizer = AdamW(model.parameters(),
                  lr = 3e-5, # args.learning_rate - default is 5e-5, our notebook had 2e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                )

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [81]:
from transformers import get_linear_schedule_with_warmup

# Number of training epochs. The BERT authors recommend between 2 and 4. 
# We chose to run for 4, but we'll see later that this may be over-fitting the
# training data.
epochs = 4

# Total number of training steps is [number of batches] x [number of epochs]. 
# (Note that this is not the same as the number of training samples).
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)

In [82]:
import numpy as np
import random
seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [83]:
def fit(epochs,model,train_dl,valid_dl,opt):
    mb = master_bar(range(epochs))
    mb.write(['epoch','train_loss','valid_loss','trn_acc','val_acc'],table=True)

    for i in mb:    
        trn_loss,val_loss = 0.0,0.0
        trn_acc,val_acc = 0,0
        trn_n,val_n = len(train_dl.dataset),len(valid_dl.dataset)
        model.train()
        for input_ids,mask,labels in progress_bar(train_dl,parent=mb):
            input_ids,mask,labels = input_ids.to(device), mask.to(device),labels.to(device)
            out = model(input_ids, 
                       token_type_ids=None, 
                       attention_mask=mask, 
                       labels=labels,
                       return_dict=True)
            opt.zero_grad()
            loss = out[0]
            _,pred = torch.max(out[1], 1)
            trn_acc += (pred == labels).sum().item()
            trn_loss += loss.item()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            opt.step()
            scheduler.step()
        trn_loss /= mb.child.total
        trn_acc /= trn_n

        model.eval()
        with torch.no_grad():
            for input_ids,mask,labels in progress_bar(valid_dl,parent=mb):
                input_ids,mask,labels = input_ids.to(device), mask.to(device),labels.to(device)
                result = model(input_ids, 
                           token_type_ids=None, 
                           attention_mask=mask,
                           labels=labels,
                           return_dict=True)
                loss = result.loss
                logits = result.logits
                val_loss += loss.item()
                _,pred = torch.max(logits, 1)
                val_acc += (pred == labels).sum().item()
        val_loss /= mb.child.total
        val_acc /= val_n

        mb.write([i,f'{trn_loss:.6f}',f'{val_loss:.6f}',f'{trn_acc:.6f}',f'{val_acc:.6f}'],table=True)

In [84]:
fit(3,model=model,train_dl=train_dataloader,valid_dl=validation_dataloader,opt=optimizer)

epoch,train_loss,valid_loss,trn_acc,val_acc
0,0.820737,0.609056,0.628797,0.737778
1,0.545090,0.598580,0.775253,0.760000
2,0.366263,0.649606,0.864658,0.777778


In [85]:
import os

# Saving best-practices: if you use defaults names for the model, you can reload it using from_pretrained()

output_dir = './model_save/'

# Create output directory if needed
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

print("Saving model to %s" % output_dir)

# Save a trained model, configuration and tokenizer using `save_pretrained()`.
# They can then be reloaded using `from_pretrained()`
model_to_save = model.module if hasattr(model, 'module') else model  # Take care of distributed/parallel training
model_to_save.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

Saving model to ./model_save/


('./model_save/tokenizer_config.json',
 './model_save/special_tokens_map.json',
 './model_save/vocab.txt',
 './model_save/added_tokens.json')